# Esercizio 4 - Soluzione

### 1) Parametri in input

In [ ]:
embl_file_name = './M10051.embl'
genetic_code_name = './genetic-code.txt'

### 2) Importazione del modulo `re`

In [ ]:
import re

### 3) Definizione di una funzione per formattare in `FASTA` una sequenza

La funzione prende come argomenti un *header* `FASTA` (che inizia con un simbolo `>`), una sequenza nucleotidica (o di proteina) e la dimensione (passata come *keyword* `record` con valore di *default* pari a 80) dei *record* in cui separare la sequenza. La funzione restituisce una stringa contenente la sequenza in formato `FASTA`.

*Suggerimento*: usare la funzione `compile()` per creare un'espressione regolare.

In [ ]:
def format_fasta(header, sequence, record = 80):    
    p = re.compile('\w{,' + str(record) + '}')
    return header + '\n' + '\n'.join(re.findall(p, sequence))

### 4) Lettura del file del codice genetico in una lista di righe

In [ ]:
with open(genetic_code_name, 'r') as genetic_file:
    genetic_code_rows = genetic_file.readlines()

In [ ]:
genetic_code_rows

### 5) Costruzione del dizionario del codice genetico

Costruire il dizionario che contiene il codice genetico:

- *chiave*: codone
- *valore*: simbolo dell'amminoacido codificato

a) Costruire la lista delle tuple *(chiave, valore)*.

*Suggerimento*: usare la funzione `product()` del modulo `itertools` che effettua il prodotto cartesiano tra le liste passate come argomento.

In [ ]:
import itertools

list(itertools.product(['a', 'b', 'c'], [1,2,3]))

In [ ]:
split_genetic_code = [row.rstrip().split(',') for row in genetic_code_rows]

In [ ]:
split_genetic_code

In [ ]:
key_value_list = []

for record_list in split_genetic_code:
    record_codon_list = record_list[1:]
    record_ammino_list = record_list[0]
    cartesian_list = list(itertools.product(record_codon_list, record_ammino_list))
    key_value_list.extend(cartesian_list)

In [ ]:
key_value_list

b) Costruire il dizionario.

In [ ]:
genetic_code_dict = dict(key_value_list)

In [ ]:
genetic_code_dict

### 6) Lettura del file `EMBL` in un'unica stringa

*Suggerimento*: leggere il file in un'unica stringa usando il metodo `read()`.

In [ ]:
with open(embl_file_name, 'r') as embl_file:
    embl_str = embl_file.read()

In [ ]:
print(embl_str)

### 7) Estrazione dell'identificatore univoco e dell'organismo relativo all'entry.

Estrarre dal *record* `ID`:

    ID   M10051; SV 1; linear; mRNA; STD; HUM; 4723 BP.
    
l'identificatore univoco `M10051` e l'organismo `HUM`.

In [ ]:
m = re.search(r'^ID\s+(\w+).+\s+(\w+);', embl_str)
(identifier, organism) = m.groups()

In [ ]:
identifier

In [ ]:
organism

### 8) Estrazione della sequenza nucleotidica

a) Ottenere la lista dei *record* della sequenza nucleotidica, dopo avere escluso da ognuno gli spazi iniziali, gli spazi finali compreso l'intero.

    ggggggctgc gcggccgggt cggtgcgcac acgagaagga cgcgcggccc ccagcgctct        60

In [ ]:
seq_row_list = re.findall('^\s+(.+?)\s+\d+', embl_str, re.M)

In [ ]:
seq_row_list

b) Concatenare le singole parti di sequenza (di 10 basi) per ottenere la sequenza nucleotidica in unica stringa senza spazi e in lettere maiuscole.

*Suggerimento*: il metodo `join()` degli oggetti `str` restituisce la concatenazione delle stringhe presenti nella lista passata come argomento usando la stringa invocante come separatore.

In [ ]:
nucleotide_sequence = ''.join(seq_row_list).replace(' ', '')

In [ ]:
nucleotide_sequence

### 9) Estrazione della sequenza della proteina

a) Estrarre il prefisso della proteina che è contenuto nel *record*:

    FT                   /translation="MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLT

In [ ]:
s = re.search('^FT\s+/translation=\"(\w+)', embl_str, re.M)
protein_prefix = s.group(1)

In [ ]:
protein_prefix

b) Costruire la lista di tutti gli altri record della proteina (compreso l'ultimo):

    FT                   RLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFP
    
**Attenzione all'ultimo**:

    FT                   DGGSSLGFKRSYEEHIPYTHMNGGKKNGRILTLPRSNPS"

**NB:** l'ultima termina con doppi apici `"`.

In [ ]:
protein_list = re.findall('^FT\s+([A-Z]+)"?$', embl_str, re.M)

In [ ]:
protein_list

b) Aggiungere in testa alla lista il prefisso trovato prima e concatenare tutti gli elementi della lista per ottenere la sequenza della proteina in un'unica stringa.

In [ ]:
protein_list[:0] = [protein_prefix]
protein_sequence = ''.join(protein_list)

In [ ]:
protein_sequence

### 10) Determinazione della coding sequence (CDS)

a) Estrarre dal *record*

    FT   CDS             139..4287
    
lo start e l'end (1-based) della CDS.

In [ ]:
(cds_start, cds_end) = tuple(map(int, re.search(r'^FT\s+CDS\s+(\d+)..(\d+)', embl_str, re.M).groups()))

In [ ]:
cds_start

In [ ]:
cds_end

b) Estrarre la sequenza della CDS.

In [ ]:
cds_sequence = nucleotide_sequence[cds_start-1:cds_end]

In [ ]:
cds_sequence

### 11) Creazione della coding sequence (CDS) in formato `FASTA`

Produrre la sequenza della CDS in formato `FASTA` (separata in *record* di 60bp) con il seguente *header*:

    >M10051-HUM; len = [length]; start = [yes|no]; end = [yes|no]
    
e assegnarla alla variabile `cds_sequence_fasta`.

In [ ]:
header = '>' + identifier + '-' + organism + '; len = ' + str(len(cds_sequence)) + ';'

exist_start_codon = 'no'
exist_stop_codon = 'no'

if cds_sequence[:3] == 'atg':
    exist_start_codon = 'yes'
    
if cds_sequence[-3:] in ['taa', 'tag', 'tga']:
    exist_stop_codon = 'yes'
    
header = header + ' start = ' + exist_start_codon + ';'
header = header + ' stop = ' + exist_stop_codon

cds_sequence_fasta = format_fasta(header, cds_sequence, record = 60)

In [ ]:
print(cds_sequence_fasta)

Stampare la coding sequence in un file.

In [ ]:
with open('./cds.fa', 'w') as output_file:
    output_file.write(cds_sequence_fasta)

### 12) Determinazione delle frequenze dei codoni

a) Estrarre la lista dei codoni della CDS.

In [ ]:
codon_list = re.findall(r'.{3}', cds_sequence)

In [ ]:
codon_list

b) Costruire la lista di tuple *(codone, frequenza)* elencate per frequenze decrescenti.

In [ ]:
from collections import Counter

codon_frequency = Counter(codon_list).most_common()

In [ ]:
codon_frequency

### 13) Determinazione delle frequenze degli amminoacidi della proteina letta dal file `EMBL`

a) Estrarre la lista degli amminoacidi della proteina.

In [ ]:
#re.findall(r'.', protein_sequence)
ammino_list = list(protein_sequence)

In [ ]:
ammino_list

a) Costruire la lista di tuple *(amminoacido, frequenza)* elencate per frequenza decrescente.

In [ ]:
ammino_frequency = Counter(ammino_list).most_common()

In [ ]:
ammino_frequency

b) Produrre il diagramma a barre delle frequenze degli amminoacidi.

Importare il package `matplotlib`

In [ ]:
import matplotlib

In [ ]:
help(matplotlib)

Importare il modulo `pyplot`.

In [ ]:
from matplotlib import pyplot

In [ ]:
help(pyplot)

Creare la lista dei simboli di amminoacido (asse x del diagramma)

In [ ]:
ammino_acid_list = [aa for (aa, f) in ammino_frequency]

In [ ]:
ammino_acid_list

Creare la corrispondente lista delle frequenze (asse y del diagramma)

In [ ]:
freq_list = [f for (aa, f) in ammino_frequency]

In [ ]:
freq_list

In [ ]:
pyplot.bar(ammino_acid_list, freq_list, color = 'purple')
pyplot.show()

### 14) Validazione della sequenza della proteina letta dal file `EMBL`

a) Tradurre in proteina la sequenza della CDS.

**Alternativa1:** traduzione della lista `codon_list` da lista di codoni a lista di amminoacidi e unione in unica stringa con il metodo `join()`.

In [ ]:
cds_translation = ''.join(genetic_code_dict[codon] for codon in codon_list[:-1])

In [ ]:
cds_translation

**Alternativa2:** traduzione dalla stringa della coding sequence con la funzione `sub()` del modulo `re`.

La funzione:

    re.sub(my_expr, r_arg, my_string)
    
sostituisce tutte le occorrenze non sovrapposte della *RE* `my_expr` in `my_string` con l'argomento `r_arg`.

In [ ]:
cds_translation2 = re.sub(r'.{3}', lambda x: genetic_code_dict[x.group()], cds_sequence[:-3])

In [ ]:
cds_translation2

Verificare che le due alternative portano alla stessa traduzione.

In [ ]:
cds_translation == cds_translation2

Verificare infine che la proteina letta dal file `EMBL` è uguale a quella ottenuta per traduzione della CDS.

In [ ]:
protein_sequence == cds_translation

### 15) Trovare una CDS "sinonima" della precedente ottenuta sostituendo il maggior numero di codoni

a) Costruire il dizionario inverso del codice genetico:

- *chiave*: simbolo di amminoacido
- *valore*: lista dei codoni che corrispondono all'amminoacido

In [ ]:
split_genetic_code

In [ ]:
key_value_list = [(record_list[0], record_list[1:]) for record_list in split_genetic_code]

In [ ]:
inverse_genetic_code_dict = dict(key_value_list)

In [ ]:
inverse_genetic_code_dict

b) Determinare una CDS sinonima sostituendo il maggior numero possibile di codoni.

In [ ]:
syn_cds_sequence = ''

for codon in codon_list:
    aa = genetic_code_dict[codon]
    
    copy_list = list(inverse_genetic_code_dict[aa])
    
    #Il codone da aggiungere alla cds sinonima è inizialmente il codone codon
    codon_to_append = codon
    if len(copy_list) > 1:
        copy_list.remove(codon)
        codon_to_append = copy_list.pop(0)
    
    syn_cds_sequence = syn_cds_sequence + codon_to_append

In [ ]:
syn_cds_sequence

c) Verificare che fornisca la stessa proteina.

In [ ]:
cds_translation2 == re.sub(r'.{3}', lambda x: genetic_code_dict[x.group()], syn_cds_sequence[:-3])

d) Misurare la differenza con la precedente CDS tramite distanza di Hamming, che è il numero di posizioni in cui le due sequenze hanno diverso carattere.

**Esempio**: la distanza di Hamming di `ACGTG` e `GCTTG` è pari a 2, in quanto le basi diverse sono quelle in posizione 1 (`A` e `G`) e quelle in posizione 3 (`G` e `T`).

*Suggerimento*: usare la funzione `range()` passando come unico argomento la lunghezza della CDS, per produrre il range dei suoi indici di posizione. 

In [ ]:
hamming_dist = [syn_cds_sequence[i] == cds_sequence[i] for i in range(len(cds_sequence))].count(False)

In [ ]:
hamming_dist

Percentuale di basi differenti rispetto alla lunghezza delle due CDS:

In [ ]:
hamming_dist / len(cds_sequence)

Numero di possibili CDS che esprimono la stessa proteina:

In [ ]:
count = 1
for aa in cds_translation:
    count = count * len(inverse_genetic_code_dict[aa])

In [ ]:
count